In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/kubig_financial/finrl")

# ## install required packages
# !pip install gym
# !pip install stockstats
# !pip install gymnasium
# !pip install stable_baselines3
# !pip install alpaca_trade_api
# !pip install exchange_calendars
# !pip install pyfolio
# !pip install matplotlib
# !pip install swig
# !pip install wrds
# !pip install pyportfolioopt
# !pip install --upgrade ipykernel
# ## install finrl library
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig


import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import itertools

from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2023-07-01 10:24:20.030690: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-01 10:24:24.063382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.10/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
%cd /content/drive/MyDrive/kubig_financial
!ls
import dlinear_module
import finrl_module

/content/drive/MyDrive/kubig_financial
dataset		      finrl		  prev	       resultsa2c
datasets	      finrl_module.ipynb  __pycache__  tensorboard_log
Dlinear_module.ipynb  finrl_module.py	  result       trained_models
dlinear_module.py     main_test.ipynb	  results      trials
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
if __name__=="__main__":
  ############# parameter ####################

  # TRADE_END_DATE = '2023-07-01' # 이거 사용안함(module안에 녹임)

  strategy = ['a2c','ddpg','ppo','td3','sac']
  using = strategy[0]

  tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY'] # 포트폴리오 구성종목
  # default로 하면 model load하고, tickers 고치면 다시 training한다.


  epochs = 200 # trade-off 이거 크면 dlinear training 시간 많이 걸림


  earning_rate, cash, record = finrl_module.finrl_main(tickers, using, epochs, FORCAST_SIZE=30, tolerance=0.0001, hmax=10000, initial_amount=1000000)

print(earning_rate,'%')
record.loc[~(record == 0).all(axis=1)]

start_date :  2009-01-01 end_date :  2023-07-01 tickers :  ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (32832, 8)
prediction_start_index 252
test_start_index 3209
3209 691
2927 409 3366
test_loader 13
torch.Size([32, 252, 9])
252
30


100%|██████████| 13/13 [00:00<00:00, 58.90it/s]


Test Loss: 707.463135
torch.Size([409, 30, 9])


100%|██████████| 106/106 [00:01<00:00, 72.23it/s]


Test Loss: 415.157074
torch.Size([3366, 30, 9])
end
['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3647, 8)
Successfully added vix
Successfully added turbulence index
Stock Dimension: 9, State Space: 118
{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to resultsa2c
----------------------------------------
| time/                 |              |
|    fps                | 163          |
|    iterations         | 100          |
|    time_elapsed       | 3            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -12.8        |
|    explained_variance | -10.9        |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | -0.164       |
|    reward             | -0.02

/usr/local/lib/python3.10/dist-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


a2c 로 얻은 투자 수익률 >> 3.06 %
3.06 %


,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
date,,,,,,,,,
2021-10-01,0,0,9464,0,0,0,1586,0,1073
2021-10-04,0,0,9943,0,0,0,0,0,0
2021-10-05,0,0,-10000,0,0,7084,-1586,0,0
2021-10-06,0,0,-9407,0,0,2,0,0,1966


In [4]:
record

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
date,,,,,,,,,
2021-10-01,0,0,9464,0,0,0,1586,0,1073
2021-10-04,0,0,9943,0,0,0,0,0,0
2021-10-05,0,0,-10000,0,0,7084,-1586,0,0
2021-10-06,0,0,-9407,0,0,2,0,0,1966
2021-10-07,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2023-06-22,0,0,0,0,0,0,0,0,0
2023-06-23,0,0,0,0,0,0,0,0,0
2023-06-26,0,0,0,0,0,0,0,0,0
